# Đồ án Xác suất thống kê chuyên sâu (DS101)
- Họ và tên: Thái Bình Dương
- MSSV: 23520356

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pandas as pd
import scipy.stats as stats

np.random.seed(42)

## Bài toán mô phỏng đá phạt đền

Bộ dữ liệu đã sử dụng: [Thống kê kết quả đá penalty ở World Cup từ 1982 (cập nhật đến 2022)](https://www.kaggle.com/datasets/pablollanderos33/world-cup-penalty-shootouts)

- Zone: Hướng sút của cầu thủ đến khung thành (được chia làm 9 phần bằng nhau, từ trái sang phải và từ trên xuống dưới)
- Keeper: Hướng đổ người của thủ môn (L là trái, C là giữa, R là bên phải, chia đều dọc theo khung thành)
- OnTarget: Có sút vào trong khung thành hay không (1 là có, 0 là không)

In [3]:
df_2 = pd.read_csv('penalty.csv')
df_2.head()

,Game_id,Team,Zone,Foot,Keeper,OnTarget,Goal,Penalty_Number,Elimination
0,1,FRA,7,R,R,1,1,1,0
1,1,GER,9,R,C,1,1,2,0
2,1,FRA,6,R,L,1,1,3,0
3,1,GER,2,R,C,1,1,4,0
4,1,FRA,9,R,L,1,1,5,0


In [4]:
df_2 = df_2.drop(['Game_id', 'Team', 'Penalty_Number', 'Elimination', 'Foot'], axis=1)
df_2

,Zone,Keeper,OnTarget,Goal
0,7,R,1,1
1,9,C,1,1
2,6,L,1,1
3,2,C,1,1
4,9,L,1,1
...,...,...,...,...
315,8,R,1,1
316,7,L,0,0
317,7,L,1,1
318,2,L,1,1


In [5]:
penalty_shoot = df_2['Zone'].count()

penalty_shoot

320

Bộ dữ liệu gồm 320 dữ liệu

Quy ước các cú sút ra ngoài dựa theo hướng sút (Ontarget == 0)

In [56]:
shoot = pd.DataFrame()
for i in range(1, 10):
    shoot[f'{i}'] = [df_2[(df_2['Zone'] == i) & (df_2['OnTarget'] == 1)].shape[0]]
shoot['10'] = [df_2[(df_2['Zone'].isin([4, 7])) & (df_2['OnTarget'] == 0)].shape[0]]
shoot['11'] = [df_2[(df_2['Zone'].isin([1, 2, 3])) & (df_2['OnTarget'] == 0)].shape[0]]
shoot['12'] = [df_2[(df_2['Zone'].isin([6, 9])) & (df_2['OnTarget'] == 0)].shape[0]]

In [57]:
shoot = shoot.astype(float)
shoot/=penalty_shoot

PMF của hướng sút

In [58]:
shoot

,1,2,3,4,5,6,7,8,9,10,11,12
0,0.071875,0.04375,0.04375,0.125,0.065625,0.1125,0.221875,0.075,0.153125,0.03125,0.046875,0.009375


In [59]:
# Xét tỉ lệ chia theo khu vực bắt bóng của thủ môn (trái, giữa, phải, ứng với vị trí 1, 2, 3 của thủ môn)
shoot_l = shoot['1'] + shoot['4'] + shoot['7']
shoot_c = shoot['2'] + shoot['5'] + shoot['8']
shoot_r = shoot['3'] + shoot['6'] + shoot['9'] 

shoot_l, shoot_c, shoot_r

(0    0.41875
 dtype: float64,
 0    0.184375
 dtype: float64,
 0    0.309375
 dtype: float64)

In [60]:
k = 0
for i in range (12, 0, -1):
    k += shoot[f'{i}']
    shoot[f'{i}'] = 1 - k

In [61]:
shoot

,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.071875,0.115625,0.159375,0.284375,0.35,0.4625,0.684375,0.759375,0.9125,0.94375,0.990625


In [62]:
keeper = pd.DataFrame()
keeper['1'] = [df_2[df_2['Keeper']=='L'].shape[0]]
keeper['2'] = [df_2[df_2['Keeper']=='C'].shape[0]]
keeper['3'] = [df_2[df_2['Keeper']=='R'].shape[0]]

In [63]:
keeper

,1,2,3
0,151,37,130


In [64]:
keeper = keeper.astype(float)
keeper/=penalty_shoot

PMF của hướng đổ người

In [65]:
keeper

,1,2,3
0,0.471875,0.115625,0.40625


In [66]:
keeper['2']+=keeper['1']
keeper['3']=1

In [67]:
keeper

,1,2,3
0,0.471875,0.5875,1


Xây dựng chương trình giả lập

In [69]:
class PenaltySimulation():
    def __init__(self) -> None:
        self.shoot_position = []
        self.keeper_position = []
    def ShootPosition(self) -> int:
        shoot_prob = np.random.uniform()
        if shoot_prob <= 0.071875:
            return 1
        elif shoot_prob <= 0.115625:
            return 2
        elif shoot_prob <= 0.159375:
            return 3
        elif shoot_prob <= 0.284375:
            return 4
        elif shoot_prob <= 0.35:
            return 5
        elif shoot_prob <= 0.4625:
            return 6
        elif shoot_prob <= 0.684375:
            return 7
        elif shoot_prob <= 0.759375:
            return 8
        elif shoot_prob <= 0.9125:
            return 9
        elif shoot_prob <= 0.94375:
            return 10
        elif shoot_prob <= 0.990625:
            return 11
        else:
            return 12
    def KeeperPosition(self) -> int:
        keeper_prob = np.random.uniform()
        if keeper_prob < 0.471875:
            return 1
        elif keeper_prob < 0.5875:
            return 2
        else:
            return 3
    def simulation(self, num_simulations: int = 1):
        missed =0
        goal = 0
        catched = 0
        catched_l = 0 # Bị cản phá ở vị trí 1
        catched_c = 0 # Bị cản phá ở vị trí 2
        catched_r = 0 # Bị cản phá ở vị trí 3
        for j in range(num_simulations):
            print(f'Lần sút phạt thứ {j+1}')
            self.shoot_position = self.ShootPosition()
            self.keeper_position = self.KeeperPosition()
            print(f'Vị trí cú sút: {self.shoot_position}, Vị trí thủ môn: {self.keeper_position}')
            if (self.shoot_position == 10 or self.shoot_position == 11 or self.shoot_position == 12):
                print('Sút ra ngoài')
                missed = missed + 1
            # Xét trường hợp bị cản phá ở bên trái
            elif ((self.shoot_position == 1 or self.shoot_position == 4 or self.shoot_position == 7) and self.keeper_position == 1):
                print('Bị cản phá')
                catched = catched + 1
                catched_l = catched_l + 1
            # Xét trường hợp bị cản phá ở chính giữa
            elif ((self.shoot_position == 2 or self.shoot_position == 5 or self.shoot_position == 8) and self.keeper_position == 2):
                print('Bị cản phá')
                catched = catched + 1
                catched_c = catched_c + 1
            # Xét trường hợp bị cản phá ở bên phải
            elif ((self.shoot_position == 3 or self.shoot_position == 6 or self.shoot_position == 9) and self.keeper_position == 3):
                print('Bị cản phá')
                catched = catched + 1
                catched_r = catched_r + 1
            else:
                print('Ghi bàn')
                goal = goal + 1
        print(f'Kết thúc giả lập có {missed} cú sút ra ngoài, {catched} cú sút bị cản phá, {goal} cú sút ghi bàn')
        print(f'Có {catched_l} cú sút bị cản phá ở bên trái, {catched_c} cú sút bị cản phá ở giữa, {catched_r} cú sút bị cản phá ở bên phải')


In [71]:
pen = PenaltySimulation()
pen.simulation(10)      #Giả lập trước với 10 lần sút

Lần sút phạt thứ 1
Vị trí cú sút: 11, Vị trí thủ môn: 1
Sút ra ngoài
Lần sút phạt thứ 2
Vị trí cú sút: 7, Vị trí thủ môn: 2
Ghi bàn
Lần sút phạt thứ 3
Vị trí cú sút: 10, Vị trí thủ môn: 1
Sút ra ngoài
Lần sút phạt thứ 4
Vị trí cú sút: 7, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 5
Vị trí cú sút: 9, Vị trí thủ môn: 2
Ghi bàn
Lần sút phạt thứ 6
Vị trí cú sút: 9, Vị trí thủ môn: 3
Bị cản phá
Lần sút phạt thứ 7
Vị trí cú sút: 1, Vị trí thủ môn: 1
Bị cản phá
Lần sút phạt thứ 8
Vị trí cú sút: 1, Vị trí thủ môn: 1
Bị cản phá
Lần sút phạt thứ 9
Vị trí cú sút: 7, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 10
Vị trí cú sút: 9, Vị trí thủ môn: 1
Ghi bàn
Kết thúc giả lập có 2 cú sút ra ngoài, 3 cú sút bị cản phá, 5 cú sút ghi bàn
Có 2 cú sút bị cản phá ở bên trái, 0 cú sút bị cản phá ở giữa, 1 cú sút bị cản phá ở bên phải


Tiếp tục giả lập với 50, 100, 1000 cú sút

In [72]:
pen.simulation(50)

Lần sút phạt thứ 1
Vị trí cú sút: 2, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 2
Vị trí cú sút: 7, Vị trí thủ môn: 1
Bị cản phá
Lần sút phạt thứ 3
Vị trí cú sút: 9, Vị trí thủ môn: 3
Bị cản phá
Lần sút phạt thứ 4
Vị trí cú sút: 6, Vị trí thủ môn: 3
Bị cản phá
Lần sút phạt thứ 5
Vị trí cú sút: 6, Vị trí thủ môn: 3
Bị cản phá
Lần sút phạt thứ 6
Vị trí cú sút: 7, Vị trí thủ môn: 1
Bị cản phá
Lần sút phạt thứ 7
Vị trí cú sút: 10, Vị trí thủ môn: 1
Sút ra ngoài
Lần sút phạt thứ 8
Vị trí cú sút: 8, Vị trí thủ môn: 1
Ghi bàn
Lần sút phạt thứ 9
Vị trí cú sút: 1, Vị trí thủ môn: 1
Bị cản phá
Lần sút phạt thứ 10
Vị trí cú sút: 7, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 11
Vị trí cú sút: 5, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 12
Vị trí cú sút: 8, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 13
Vị trí cú sút: 7, Vị trí thủ 

In [73]:
pen.simulation(100)

Lần sút phạt thứ 1
Vị trí cú sút: 4, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 2
Vị trí cú sút: 7, Vị trí thủ môn: 1
Bị cản phá
Lần sút phạt thứ 3
Vị trí cú sút: 4, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 4
Vị trí cú sút: 8, Vị trí thủ môn: 1
Ghi bàn
Lần sút phạt thứ 5
Vị trí cú sút: 5, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 6
Vị trí cú sút: 8, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 7
Vị trí cú sút: 6, Vị trí thủ môn: 3
Bị cản phá
Lần sút phạt thứ 8
Vị trí cú sút: 6, Vị trí thủ môn: 1
Ghi bàn
Lần sút phạt thứ 9
Vị trí cú sút: 7, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 10
Vị trí cú sút: 5, Vị trí thủ môn: 1
Ghi bàn
Lần sút phạt thứ 11
Vị trí cú sút: 7, Vị trí thủ môn: 2
Ghi bàn
Lần sút phạt thứ 12
Vị trí cú sút: 9, Vị trí thủ môn: 3
Bị cản phá
Lần sút phạt thứ 13
Vị trí cú sút: 1, Vị trí thủ môn: 3
Ghi bàn
Lần s

In [74]:
pen.simulation(1000)

Lần sút phạt thứ 1
Vị trí cú sút: 5, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 2
Vị trí cú sút: 10, Vị trí thủ môn: 3
Sút ra ngoài
Lần sút phạt thứ 3
Vị trí cú sút: 2, Vị trí thủ môn: 1
Ghi bàn
Lần sút phạt thứ 4
Vị trí cú sút: 9, Vị trí thủ môn: 1
Ghi bàn
Lần sút phạt thứ 5
Vị trí cú sút: 11, Vị trí thủ môn: 3
Sút ra ngoài
Lần sút phạt thứ 6
Vị trí cú sút: 2, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 7
Vị trí cú sút: 6, Vị trí thủ môn: 3
Bị cản phá
Lần sút phạt thứ 8
Vị trí cú sút: 6, Vị trí thủ môn: 1
Ghi bàn
Lần sút phạt thứ 9
Vị trí cú sút: 7, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 10
Vị trí cú sút: 1, Vị trí thủ môn: 3
Ghi bàn
Lần sút phạt thứ 11
Vị trí cú sút: 4, Vị trí thủ môn: 1
Bị cản phá
Lần sút phạt thứ 12
Vị trí cú sút: 5, Vị trí thủ môn: 1
Ghi bàn
Lần sút phạt thứ 13
Vị trí cú sút: 5, Vị trí thủ môn: 1
Ghi ba

## Bài toán kiểm định độc lập giữa kết quả thắng thua của đội bóng ghi bàn mở tỷ số với giải đấu

Khảo sát tất cả các trận đấu có bàn thắng tại 4 giải đấu hàng đầu châu Âu thu được như sau:

<table border="1" style="border-collapse: collapse; text-align: left;">
    <thead>
        <tr>
            <th style="text-align: center;">Kết quả</th>
            <th><strong>EPL</strong></th>
            <th><strong>La Liga</strong></th>
            <th><strong>Serie A</strong></th>
            <th><strong>Bundesliga</strong></th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><strong>Đội bóng mở tỷ số thắng trận</strong></td>
            <td>236</td>
            <td>241</td>
            <td>222</td>
            <td>189</td>
        </tr>
        <tr>
            <td><strong>Đội bóng mở tỷ số bị gỡ hoà</strong></td>
            <td>70</td>
            <td>75</td>
            <td>82</td>
            <td>68</td>
        </tr>
        <tr>
            <td><strong>Đội bóng mở tỷ số thua trận</strong></td>
            <td>59</td>
            <td>38</td>
            <td>36</td>
            <td>37</td>
        </tr>
    </tbody>
</table>


Gọi

$H_0$: Kết quả thắng thua của đội bóng ghi bàn mở tỷ số độc lập với giải đấu

$H_1$: Kết quả thắng thua của đội bóng ghi bàn mở tỷ số phụ thuộc vào giải đấu

In [ ]:
# Dữ liệu tần suất quan sát
observed = np.array([[236, 241, 222, 189],
                     [70,  75,  82,  68],
                     [59,  38,  36,  37]])

# Tổng số hàng, cột và toàn bảng
row_totals = observed.sum(axis=1)
col_totals = observed.sum(axis=0)
grand_total = observed.sum()

# Tính tần suất kỳ vọng
expected = np.outer(row_totals, col_totals) / grand_total

# Tính giá trị chi bình phương (Q)
chi2_stat = ((observed - expected) ** 2 / expected).sum()

# Bậc tự do (df)
df = (observed.shape[0] - 1) * (observed.shape[1] - 1)

# Giá trị tới hạn và p-value
critical_value = stats.chi2.ppf(0.95, df)  # Mức ý nghĩa 5%
p_value = 1 - stats.chi2.cdf(chi2_stat, df)


print(expected)
print("Bậc tự do:", df)
print("Giá trị tới hạn:", critical_value)
print("Chi-Square:", chi2_stat)
if (chi2_stat >= critical_value):
    print("Bác bỏ giả thuyết")
else:
    print("Chấp nhận giả thuyết")

[[239.55654102 232.33702882 223.14855876 192.9578714 ]
 [ 79.58240946  77.18403548  74.1315595   64.10199557]
 [ 45.86104952  44.4789357   42.71988174  36.94013304]]
Bậc tự do: 6
Giá trị tới hạn: 12.591587243743977
Chi-Square: 8.5158376178825
Chấp nhận giả thuyết


## Bài toán dự đoán xác suất đội bóng thắng, hoà hoặc thua trong trận đấu tiếp theo

Đối tượng được thử nghiệm: Câu lạc bộ Manchester United

Bộ dữ liệu được sử dụng: [Kết quả trận đấu giải Ngoại hạng Anh](https://www.kaggle.com/datasets/mertbayraktar/english-premier-league-matches-20232024-season)

In [37]:
df = pd.read_csv('epl.csv')
df.head()

,Unnamed: 0,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,...,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,2024,ManchesterCity
1,3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,...,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,2024,ManchesterCity
2,4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,...,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,2024,ManchesterCity
3,5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,...,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,2024,ManchesterCity
4,6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,...,Match Report,NaN,29.0,13.0,16.4,1.0,0,0,2024,ManchesterCity


In [41]:
problem_3 = df[df['Team'] == 'ManchesterUnited']

In [42]:
problem_3

,Unnamed: 0,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
266,0,2023-08-14,20:00,Premier League,Matchweek 1,Mon,Home,W,1,0,...,Match Report,NaN,15.0,2.0,14.7,0.0,0,0,2024,ManchesterUnited
267,1,2023-08-19,17:30,Premier League,Matchweek 2,Sat,Away,L,0,2,...,Match Report,NaN,22.0,6.0,15.6,1.0,0,0,2024,ManchesterUnited
268,2,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,W,3,2,...,Match Report,NaN,17.0,8.0,20.3,0.0,1,1,2024,ManchesterUnited
269,3,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Away,L,1,3,...,Match Report,NaN,10.0,2.0,16.5,0.0,0,0,2024,ManchesterUnited
270,4,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Home,L,1,3,...,Match Report,NaN,14.0,4.0,18.6,1.0,0,0,2024,ManchesterUnited
271,6,2023-09-23,20:00,Premier League,Matchweek 6,Sat,Away,W,1,0,...,Match Report,NaN,11.0,4.0,15.0,0.0,0,0,2024,ManchesterUnited
272,8,2023-09-30,15:00,Premier League,Matchweek 7,Sat,Home,L,0,1,...,Match Report,NaN,19.0,3.0,15.5,0.0,0,0,2024,ManchesterUnited
273,10,2023-10-07,15:00,Premier League,Matchweek 8,Sat,Home,W,2,1,...,Match Report,NaN,21.0,8.0,18.3,1.0,0,0,2024,ManchesterUnited
274,11,2023-10-21,20:00,Premier League,Matchweek 9,Sat,Away,W,2,1,...,Match Report,NaN,14.0,5.0,20.6,1.0,0,0,2024,ManchesterUnited
275,13,2023-10-29,15:30,Premier League,Matchweek 10,Sun,Home,L,0,3,...,Match Report,NaN,7.0,3.0,15.2,0.0,0,0,2024,ManchesterUnited


In [43]:
states = problem_3['Result']

states

266    W
267    L
268    W
269    L
270    L
271    W
272    L
273    W
274    W
275    L
276    W
277    W
278    W
279    L
280    W
281    L
282    D
283    L
284    W
285    L
286    D
287    W
288    W
289    W
290    W
291    L
292    L
293    W
294    D
295    L
296    D
297    D
298    W
299    D
300    L
301    L
302    W
303    W
Name: Result, dtype: object

In [44]:
# Đếm số lượng trận thắng, hoà, thua
result_count = pd.DataFrame(states.value_counts(), index=['W', 'D', 'L'])

In [45]:
result_count

,count
W,18
D,6
L,14


In [46]:
# Chia tỷ lệ với tổng số trận đấu và chuyển về ma trận
result_count = result_count/38

In [47]:
result_count = result_count.__array__()
result_count = result_count.T

In [48]:
print(result_count)

[[0.47368421 0.15789474 0.36842105]]


Xây dựng ma trận chuyển đổi trạng thái như sau

$
P =
\begin{bmatrix}
p_{00} & p_{01} & p_{02} \\
p_{10} & p_{11} & p_{12} \\
p_{20} & p_{21} & p_{22}
\end{bmatrix}
$

Với 0 là thắng (W), 1 là hoà (D), 2 là thua (L)

In [26]:
# Gắn nhãn W = 0, D = 1, L = 2
states = states.replace(['W', 'D', 'L'], [0, 1, 2])
states = np.array(states)
states

C:\Users\harry\AppData\Local\Temp\ipykernel_10468\3219450631.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  states = states.replace(['W', 'D', 'L'], [0, 1, 2])


array([0, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 1, 2, 0, 2, 1, 0,
       0, 0, 0, 2, 2, 0, 1, 2, 1, 1, 0, 1, 2, 2, 0, 0], dtype=int64)

In [27]:
# Số trạng thái: W, D, L
num_states = 3

# Đếm chuyển trạng thái
transition_counts = np.zeros((num_states, num_states))

for i in range(len(states) - 1):
    current_state = states[i]
    next_state = states[i + 1]
    transition_counts[current_state, next_state] += 1

# Tính ma trận xác suất chuyển trạng thái
transition_matrix = transition_counts / transition_counts.sum(axis=1, keepdims=True)

print(transition_matrix)


[[0.41176471 0.11764706 0.47058824]
 [0.33333333 0.16666667 0.5       ]
 [0.57142857 0.21428571 0.21428571]]


### Cách 1: Xét 3 trường hợp trạng thái ban đầu (trận đấu trước đó) thắng, hoà hoặc thua

In [28]:
if_win = [1, 0, 0]
if_draw = [0, 1, 0]
if_lose = [0, 0, 1]

print("Xác suất nếu trận trước đó là trận thắng:")
for i in range(38):
    if_win = if_win @ transition_matrix
    print(f'Trận đấu thứ {i+1}', if_win)

print("Xác suất nếu trận trước đó là trận hoà:")
for i in range(38):
    if_draw = if_draw @ transition_matrix
    print(f'Trận đấu thứ {i+1}', if_draw)

print("Xác suất nếu trận trước đó là trận thua:")
for i in range(38):
    if_lose = if_lose @ transition_matrix
    print(f'Trận đấu thứ {i+1}', if_lose)

Xác suất nếu trận trước đó là trận thắng:
Trận đấu thứ 1 [0.41176471 0.11764706 0.47058824]
Trận đấu thứ 2 [0.47767342 0.16889109 0.35343549]
Trận đấu thứ 3 [0.45494922 0.16008156 0.38496921]
Trận đấu thứ 4 [0.46067496 0.1626971  0.37662794]
Trận đấu thứ 5 [0.45913802 0.16201922 0.37884275]
Trận đấu thứ 6 [0.45954481 0.16220003 0.37825515]
Trận đấu thứ 7 [0.45943681 0.16215211 0.37841108]
Trận đấu thứ 8 [0.45946547 0.16216483 0.3783697 ]
Trận đấu thứ 9 [0.45945787 0.16216145 0.37838068]
Trận đấu thứ 10 [0.45945988 0.16216235 0.37837777]
Trận đấu thứ 11 [0.45945935 0.16216211 0.37837854]
Trận đấu thứ 12 [0.45945949 0.16216218 0.37837834]
Trận đấu thứ 13 [0.45945945 0.16216216 0.37837839]
Trận đấu thứ 14 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 15 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 16 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 17 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 18 [0.45945946 0.1621621

### Cách 2: Xét trường hợp trạng thái ban đầu là tính theo tỉ lệ giữa kết quả chung cuộc với tổng số trận đấu

In [49]:
for i in range(38):
    result_count = result_count @ transition_matrix
    print(f'Trận đấu thứ {i+1}', result_count)

Trận đấu thứ 1 [[0.45820433 0.16099071 0.38080495]]
Trận đấu thứ 2 [[0.45993877 0.16233924 0.37772199]]
Trận đấu thứ 3 [[0.45934077 0.16210741 0.37855182]]
Trận đấu thứ 4 [[0.45949145 0.16217624 0.37833231]]
Trận đấu thứ 5 [[0.459451  0.1621584 0.3783906]]
Trận đấu thứ 6 [[0.45946171 0.16216316 0.37837514]]
Trận đấu thứ 7 [[0.45945886 0.1621619  0.37837924]]
Trận đấu thứ 8 [[0.45945962 0.16216223 0.37837815]]
Trận đấu thứ 9 [[0.45945942 0.16216214 0.37837844]]
Trận đấu thứ 10 [[0.45945947 0.16216217 0.37837836]]
Trận đấu thứ 11 [[0.45945946 0.16216216 0.37837838]]
Trận đấu thứ 12 [[0.45945946 0.16216216 0.37837838]]
Trận đấu thứ 13 [[0.45945946 0.16216216 0.37837838]]
Trận đấu thứ 14 [[0.45945946 0.16216216 0.37837838]]
Trận đấu thứ 15 [[0.45945946 0.16216216 0.37837838]]
Trận đấu thứ 16 [[0.45945946 0.16216216 0.37837838]]
Trận đấu thứ 17 [[0.45945946 0.16216216 0.37837838]]
Trận đấu thứ 18 [[0.45945946 0.16216216 0.37837838]]
Trậ

### Cách 3: Giả sử tỷ lệ thắng, hoà, thua là như nhau $(\frac{1}{3})$

In [50]:
pi = [1/3, 1/3, 1/3]

for i in range(38):
    pi = pi @ transition_matrix
    print(f'Trận đấu thứ {i+1}', pi)

Trận đấu thứ 1 [0.4388422  0.16619981 0.39495798]
Trận đấu thứ 2 [0.46178994 0.16396232 0.37424774]
Trận đấu thứ 3 [0.45865876 0.16185123 0.37949002]
Trận đấu thứ 4 [0.45966134 0.16225435 0.37808432]
Trận đấu thứ 5 [0.45940528 0.16213826 0.37845646]
Trận đấu thứ 6 [0.4594738  0.16216854 0.37835766]
Trận đấu thứ 7 [0.45945565 0.16216047 0.37838388]
Trận đấu thứ 8 [0.45946047 0.16216261 0.37837692]
Trận đấu thứ 9 [0.45945919 0.16216204 0.37837877]
Trận đấu thứ 10 [0.45945953 0.16216219 0.37837828]
Trận đấu thứ 11 [0.45945944 0.16216215 0.37837841]
Trận đấu thứ 12 [0.45945946 0.16216216 0.37837837]
Trận đấu thứ 13 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 14 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 15 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 16 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 17 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 18 [0.45945946 0.16216216 0.37837838]
Trận đấu thứ 19 [0.45945946 0.1621